# 📦 Inventory Health Analyzer + Reorder Alert System
This notebook analyzes inventory data from AstroDynamics and flags critical SKUs based on reorder thresholds, risk flags, and KPI logic.

In [ ]:
# ✅ Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# ✅ Step 2: Load Dataset
df = pd.read_csv('../data/AstroDynamicsCleaned_Dataset.csv')
df.head()

Next step: We will calculate Days of Supply, Reorder Triggers, and identify critical SKUs.

## 📊 Step 3: Calculate KPIs and Inventory Health Metrics

In [ ]:
# Days of Supply = CURRENT STOCK / AVG DAILY USAGE
df['DAYS_OF_SUPPLY'] = df['CURRENT STOCK'] / df['AVG DAILY USAGE']

In [ ]:
# Inventory Turnover = (AVG DAILY USAGE * 365) / AVERAGE INVENTORY
# For simplification, we approximate average inventory as CURRENT STOCK / 2
df['INVENTORY_TURNOVER'] = (df['AVG DAILY USAGE'] * 365) / (df['CURRENT STOCK'] / 2)

## 🚨 Step 4: Flag Critical Inventory

In [ ]:
# Flag if inventory is below reorder point
df['REORDER_ALERT'] = df['CURRENT STOCK'] <= df['REORDER POINT ']
df['CRITICAL_ALERT'] = (df['STATUS FLAG'] == 'CRITICAL') | (df['SUPPLY GAP FLAG'] == 'UNDERORDERED')

In [ ]:
# Filter critical SKUs
critical_df = df[df['CRITICAL_ALERT'] == True][[
    'PART NUMBER', 'PART NAME', 'CURRENT STOCK', 'REORDER POINT ',
    'SAFETY STOCK', 'DAYS_OF_SUPPLY', 'VENDOR', 'STATUS FLAG', 'SUPPLY GAP FLAG'
]]
critical_df.head()

## 📈 Step 5: Visualize Top 10 Critical Inventory by Value

In [ ]:
top_critical_by_value = df[df['CRITICAL_ALERT'] == True]
top_critical_by_value = top_critical_by_value.sort_values(by='INVENTORY VALUE', ascending=False).head(10)
plt.figure(figsize=(10,6))
sns.barplot(data=top_critical_by_value, x='INVENTORY VALUE', y='PART NAME', hue='VENDOR')
plt.title('Top 10 Critical Inventory by Value')
plt.xlabel('Inventory Value ($)')
plt.ylabel('Part Name')
plt.tight_layout()
plt.show()

## 💾 Step 6: Export Critical Alerts to Excel

In [ ]:
output_path = '../output/critical_inventory_alerts.xlsx'
critical_df.to_excel(output_path, index=False)
print(f'Critical Inventory Alerts exported to: {output_path}')